## Convert paradigm epochs to current format

In [ ]:
import pandas as pd
import numpy as np

file = "/data/Clustering/SleepDeprivation/RatN/Day2/RatN_Day2_2019-10-11_03-58-54_epochs.npy"

data = np.load(file, allow_pickle=True).item()

epochs = pd.DataFrame(data)
a = epochs.T.reset_index()

epochs_new = pd.DataFrame(
    {"start": a[0], "stop": a[1], "label": a["index"].str.lower()}
)
metadata = None

data = {"epochs": epochs_new, "metadata": metadata}

# np.save(file[:-11] + ".paradigm.npy", data)


In [ ]:
import numpy as np

file = '/data/Clustering/sessions/RatS/Day5TwoNovel/old/RatS-Day5TwoNovel-2020-12-04_07-55-09_epochs.npy'

a = np.load(file,allow_pickle=True).item()

In [ ]:
import subjects
import numpy as np

sess = subjects.sd.ratSday3[0]

len(sess.neurons)

In [1]:
from scipy.linalg import block_diag
import numpy as np
import subjects

sess = subjects.sd.ratSday3[0]

wav = sess.neurons.waveforms

a = np.arange(25).reshape(5,5)
b = np.arange(12).reshape(4,3)

block_diag(*[wav,wav])

ValueError: arguments in the following positions have dimension greater than 2: [0, 1]

In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from neuropy.plotting import plot_waveforms
import subjects

sess = subjects.sd.ratSday3[0]
shank_ids = sess.neurons.shank_ids
sort_ind = np.argsort(shank_ids)[::-1]
plot_waveforms(sess.neurons,sort_order=sort_ind,color='g')

In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from neuropy.plotting import plot_waveforms
import subjects

sess = subjects.sd.ratSday3[0]
wav = sess.neurons.waveforms

a1,b1 = wav[20],wav[25]
b1 = np.where(b1!=0,b1,np.nan)
# plot_wv = a1
corr = np.corrcoef(a1.astype('float'),b1.astype('float'))
# plt.imshow(corr[173:,:173])
# plt.plot(b1.T+np.linspace(0,500,a1.shape[0]))

In [ ]:
waveforms = sess.neurons.waveforms
# waveforms = np.where(waveforms!=0,waveforms,np.nan)
mx = np.ma.masked_equal(waveforms,value=0)
waveforms = np.ma.reshape(mx, (sess.neurons.n_neurons, -1)).astype(float)
similarity = np.ma.corrcoef(waveforms)
np.fill_diagonal(similarity, 0)

In [2]:
import numpy as np
bins = np.arange(-1,1,0.001)
hist_sim = np.histogram(np.arange(100),bins=bins)[0]

In [ ]:
%matplotlib widget
# plt.imshow(similarity[np.ix_(sort_ind,sort_ind)])
plt.plot(hist_sim)

In [ ]:
import numpy as np
import subjects
from neuropy import analyses, plotting

sess = subjects.sd.utkuAG_day2[0]

signal = sess.eegfile.get_signal(channel_indx=np.arange(24))
rpls = analyses.oscillations.detect_ripple_epochs(
    signal, sess.recinfo.channel_groups[:4], ignore_epochs=sess.artifact,mergedist=0.005,maxdur=0.600
)
rpls.filename = sess.filePrefix.with_suffix(".ripple")
rpls.save()

# signal = sess.eegfile.get_signal([1])
# artifact_epochs = analyses.detect_artifact_epochs(signal, thresh=9)
# artifact_epochs.filename = sess.filePrefix.with_suffix(".artifact.npy")
# artifact_epochs.save()
# plotting.plot_artifact_epochs(artifact_epochs, signal)


In [ ]:
sess.paradigm.to_dataframe()

In [ ]:
sess.recinfo.write_epochs(sess.ripple)

In [ ]:
%matplotlib widget

plotting.plot_artifact_epochs(artifact_epochs,signal)

## Convert artifact to current format

In [ ]:
import pandas as pd
import numpy as np

file = "/data/Clustering/SleepDeprivation/RatS/Day3SD/RatS_Day3SD_2020-11-29_07-53-30.artifact.npy"

data = np.load(file, allow_pickle=True).item()

epochs = pd.DataFrame(
    {"start": data["time"][:, 0], "stop": data["time"][:, 1], "label": ""}
)
metadata = {"channels": data["channel"], "thresh": data["threshold"]}
data = {"epochs": epochs, "metadata": metadata}

# np.save(file, data)


In [ ]:
from spikeextractors import BinDatRecordingExtractor

a = BinDatRecordingExtractor(file, sampling_frequency=1250, numchan=134, dtype="int16")
a._epochs

In [ ]:
import numpy as np

from bokeh.plotting import figure, output_file, show

output_notebook()

ramp = np.array([np.linspace(0, 10, 200)]*20)
steps = np.array([np.linspace(0, 10, 10)]*20)
bitmask = np.random.rand(25, 10) > 0.5

data = dict(image=[ramp, steps, bitmask],
            squared=[ramp**2, steps**2, bitmask**2],
            pattern=['smooth ramp', 'steps', 'bitmask'],
            x=[0, 0, 25],
            y=[5, 20, 5],
            dw=[20,  20, 10],
            dh=[10,  10, 25])

TOOLTIPS = [
    ('name', "$name"),
    ('index', "$index"),
    ('pattern', '@pattern'),
    ("x", "$x"),
    ("y", "$y"),
    ("value", "@image"),
    ('squared', '@squared')
]

p = figure(x_range=(0, 35), y_range=(0, 35), tools='hover,wheel_zoom', tooltips=TOOLTIPS)
p.image(source=data, image='image', x='x', y='y', dw='dw', dh='dh', palette="Inferno256", name="Image Glyph")

show(p)

In [ ]:
image

In [ ]:
from bokeh.plotting import ColumnDataSource, figure, output_notebook, show
import numpy as np
from PIL import Image
from io import BytesIO
from matplotlib import cm
output_notebook()
from base64 import b64decode,b64encode
import matplotlib.pyplot as plt
from skimage import img_as_ubyte

def to_png(arr):
    out = BytesIO()
    im = Image.fromarray(arr)
    im.save(out, format='png')
    return out.getvalue()

def b64_image_files(images, colormap='magma'):
    cmap = cm.get_cmap(colormap)
    urls = []
    for im in images:
        png = to_png(img_as_ubyte(cmap(im)))
        url = 'data:image/png;base64,' + b64encode(png).decode('utf-8')
        urls.append(url)
    return urls

out = BytesIO()
a = np.arange(25).reshape(5,5)
a = a/25
# encoded = b64decode(a)
# mime = "image/jpeg"

filenames = b64_image_files([image])
# uri = "data:%s;base64,%s" % (mime, encoded)
# im = plt.imshow(a)

# im = Image.fromarray(a)
# im.save(out,format='png')

# im = Image.fromarray(np.uint8(cm.gist_earth(a)*255))
# img = out.getvalue()

source = ColumnDataSource(data=dict(
    x=[0.5, 1, 1, 1, 1],
    y=[1.5, 2, 2, 2, 2],
    desc=['A', 'b', 'C', 'd', 'E'],
    imgs=[
        filenames,
        'https://docs.bokeh.org/static/snake2.png',
        'https://docs.bokeh.org/static/snake3D.png',
        'https://docs.bokeh.org/static/snake4_TheRevenge.png',
        'https://docs.bokeh.org/static/snakebite.jpg'
    ],
    fonts=[
        '<i>italics</i>',
        '<pre>pre</pre>',
        '<b>bold</b>',
        '<small>small</small>',
        '<del>del</del>'
    ]
))

TOOLTIPS = """
    <div>
        <div>
            <img
                src="@imgs" height="180" alt="@imgs" width="180"
                style="float: left; margin: 0px 15px 15px 0px;"
                border="2"
            ></img>
        </div>
        <div>
            <span style="font-size: 17px; font-weight: bold;">@desc</span>
            <span style="font-size: 15px; color: #966;">[$index]</span>
        </div>
        <div>
            <span>@fonts{safe}</span>
        </div>
        # <div>
            <span style="font-size: 15px;">Location</span>
            <span style="font-size: 10px; color: #696;">($x, $y)</span>
        </div>
    </div>
"""

p = figure(width=400, height=400, tooltips=TOOLTIPS,
           title="Mouse over the dots")

p.circle('x', 'y', size=20, source=source)

show(p)

In [ ]:
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure

fig = Figure()
width,height = fig.get_size_inches()* fig.get_dpi()
canvas = FigureCanvas(fig)
ax = fig.gca()

ax.text(0.0,0.0,"Test", fontsize=45)
ax.axis('off')

canvas.draw()       # draw the canvas, cache the renderer

image = np.frombuffer(canvas.tostring_rgb(), dtype='uint8')
image = image.reshape(height.astype(int),width.astype(int),3)[:,:,1]

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

x = [1,2,3,4]
y = [6,7,2,4]
p = figure(title='Sample line')
p.line(x,y)
show(p)

In [ ]:
import numpy as np
from pathlib import Path

file = Path("/data/Clustering/SleepDeprivation/RatN_Day1_test_neuropy/params.py")

params = {}
with file.open("r") as f:
    for line in f:
        line_values = (
            line.replace("\n", "").replace('r"', '"').replace('"', "").split("=")
        )
        params[line_values[0].strip()] = line_values[1].strip()


params

In [ ]:
import pandas as pd
from pathlib import Path

file = Path("/data/Clustering/SleepDeprivation/RatN_Day1_test_neuropy/cluster_info.tsv")
data = pd.read_csv(file, delimiter="\t")
a = data[data["group"].isin(["mua", "good"])].reset_index(drop=True)

b = np.arange(10)
for i in data.itertuples():
    np.where(b == i.id)


In [ ]:
from pathlib import Path
from neuropy.io import PhyIO
from neuropy.core import Neurons

phy_data = PhyIO(
    Path(
        "/home/bapung/Documents/ClusteringHub/spykcirc/RatR/Day2SD/Allsession/RatR_Day2SD_2021-05-23_09-07-18-1.GUI"
    )
)

neurons = Neurons(
    phy_data.spiketrains, t_stop=53000, t_start=0, sampling_rate=phy_data.sampling_rate
)


In [ ]:
import numpy as np
from neuropy.core import Position


x = np.random.randn(10)
y = np.random.randn(10)
z = np.random.randn(10)
t = np.arange(0,10)

pos = Position(t,x,y,z,sampling_rate=1)



In [ ]:
import numpy as np
from neuropy.core import Mua

mua = Mua(spike_counts=np.array([1,2,3]),bin_size=0.2)

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np

file1 = '/data/Clustering/sessions/RatS/Day2NSD/RatS-Day2NSD-2020-11-27_10-22-29_position.npy'
file2 = '/data/Clustering/sessions/RatS/Day2NSD/RatS-Day2NSD-2020-11-27_10-22-29.position.npy'

data1 = np.load(file1,allow_pickle=True).item()
data2 = np.load(file2,allow_pickle=True).item()


plt.plot(data1['x'])
plt.plot(data2['traces'][1])

In [ ]:
import numpy as np

file = '/data/Clustering/sessions/RatN/Day2/2019-10-11_03-58-54/experiment1/recording1/continuous/Rhythm_FPGA-100.0/timestamps.npy'

data = np.load(file)

In [ ]:
fileName ='/data/Clustering/sessions/RatN/Day2/position/Take 2019-10-11 04.08.48 AM.fbx'

with open(fileName) as f:
    next(f)
    for i, line in enumerate(f):

        m = "".join(line)

        if "KeyCount" in m:
            print(m)
            # print("break at i = " + str(i))
            # line_frame = linecache.getline(fileName, i + 2).strip().split(" ")
            break



In [ ]:
import numpy as np

file = '/data/Clustering/sessions/RatU/RatUDay2NSD/RatU_Day2NSD_2021-07-24_08-16-38.paradigm.npy'

data = np.load(file,allow_pickle=True).item()
data['epochs']


In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt

file_name = "RatU_Day4SD_2021-07-29_08-23-06.eeg_sample.eeg"
n_channels = 192 # number of recorded channels in the file
sampling_rate = 1250
# reading data from binary file and reshaping to n_channels x time format
data = np.memmap(file_name,dtype='int16',mode='r').reshape(-1,n_channels).T

#let's plot data from first channel
plt.plot(data[0])


In [ ]:
%matplotlib widget
import numpy as np
from neuropy.plotting import Fig

def example_plot(ax, fontsize=12, hide_labels=False):
    pc = ax.pcolormesh(np.random.randn(30, 30), vmin=-2.5, vmax=2.5)
    if not hide_labels:
        ax.set_xlabel('x-label', fontsize=fontsize)
        ax.set_ylabel('y-label', fontsize=fontsize)
        ax.set_title('Title', fontsize=fontsize)
    return pc

figure = Fig()
fig,gs = figure.draw(grid=(4,4))

ax = figure.add_subplot(gs[:,2:])
ax.plot(np.arange(10),np.arange(10))

subfig = fig.add_subfigure(gs[:,:2])
axsLeft = subfig.subplots(1, 2, sharey=True)
subfig.set_facecolor('0.75')
for ax in axsLeft:
    pc = example_plot(ax)
subfig.suptitle('Left plots', fontsize='x-large')
subfig.colorbar(pc, shrink=0.6, ax=axsLeft, location='bottom')

In [ ]:
import numpy as np

from bokeh.plotting import figure, show, output_file
from bokeh.models import HoverTool, CustomJS, ColumnDataSource

N = 20
x = np.random.random(size=N)
y = np.random.random(size=N)

p = figure(tools=["hover"], toolbar_location=None)
p.scatter(x, y, size=10)

img_x = np.linspace(0, 10, N)
img_y = np.linspace(0, 10, N)

images = []
for a, b in zip(x, y):
    xx, yy = np.meshgrid(x, y)
    d = np.sin(a*xx)*np.cos(b*yy)
    images.append(d)

imgs_source = ColumnDataSource(data=dict(images=images))
img_source = ColumnDataSource(data=dict(image=[]))

img = figure(x_range=(0, 10), y_range=(0, 10), tools=[], toolbar_location=None)
img.image(image="image", x=0, y=0, dw=10, dh=10, source=img_source, palette="Greys7")

hover = p.select_one(HoverTool)
hover.callback = CustomJS(args=dict(imgs_source=imgs_source, img_source=img_source), code="""
var indices = cb_data.index['1d'].indices;
if (indices.length > 0) {
    var img = imgs_source.data.images[indices[0]];
    img_source.data = {image: [img]};
} else {
    img_source.data = {image: []};
}
""")
hover.tooltips = None

output_file("linked_hover.html")
show(p)

In [ ]:
import base64
from math import cos, pi, sin

from bokeh.colors.named import firebrick, lightgray, orchid, seagreen, skyblue, tomato
from bokeh.document import Document
from bokeh.embed import file_html
from bokeh.models import (AnnularWedge, ColumnDataSource,
                          ImageURL, Plot, Range1d, Text, Wedge)
from bokeh.resources import INLINE
from bokeh.sampledata.browsers import browsers_nov_2013, icons
from bokeh.util.browser import view

df = browsers_nov_2013

xdr = Range1d(start=-2, end=2)
ydr = Range1d(start=-2, end=2)

plot = Plot(x_range=xdr, y_range=ydr, width=800, height=800)
plot.title.text = "Web browser market share (November 2013)"
plot.toolbar_location = None

colors = {"Chrome": seagreen, "Firefox": tomato, "Safari": orchid, "Opera": firebrick, "IE": skyblue, "Other": lightgray}

aggregated = df.groupby("Browser").agg(sum)
selected = aggregated[aggregated.Share >= 1].copy()
selected.loc["Other"] = aggregated[aggregated.Share < 1].sum()
browsers = selected.index.tolist()

radians = lambda x: 2*pi*(x/100)
angles = selected.Share.map(radians).cumsum()

end_angles = angles.tolist()
start_angles = [0] + end_angles[:-1]

browsers_source = ColumnDataSource(dict(
    start  = start_angles,
    end    = end_angles,
    colors = [colors[browser] for browser in browsers ],
))

glyph = Wedge(x=0, y=0, radius=1, line_color="white",
    line_width=2, start_angle="start", end_angle="end", fill_color="colors")
plot.add_glyph(browsers_source, glyph)

def polar_to_cartesian(r, start_angles, end_angles):
    cartesian = lambda r, alpha: (r*cos(alpha), r*sin(alpha))
    points = []

    for start, end in zip(start_angles, end_angles):
        points.append(cartesian(r, (end + start)/2))

    return zip(*points)

first = True

for browser, start_angle, end_angle in zip(browsers, start_angles, end_angles):
    versions = df[(df.Browser == browser) & (df.Share >= 0.5)]
    angles = versions.Share.map(radians).cumsum() + start_angle
    end = angles.tolist() + [end_angle]
    start = [start_angle] + end[:-1]
    base_color = colors[browser]
    fill = [ base_color.lighten(i*0.05).to_hex() for i in range(len(versions) + 1) ]
    # extra empty string accounts for all versions with share < 0.5 together
    text = [ number if share >= 1 else "" for number, share in zip(versions.VersionNumber, versions.Share) ] + [""]
    x, y = polar_to_cartesian(1.25, start, end)

    source = ColumnDataSource(dict(start=start, end=end, fill=fill))
    glyph = AnnularWedge(x=0, y=0,
        inner_radius=1, outer_radius=1.5, start_angle="start", end_angle="end",
        line_color="white", line_width=2, fill_color="fill")
    plot.add_glyph(source, glyph)

    text_angle = [(start[i]+end[i])/2 for i in range(len(start))]
    text_angle = [angle + pi if pi/2 < angle < 3*pi/2 else angle for angle in text_angle]

    text_source = ColumnDataSource(dict(text=text, x=x, y=y, angle=text_angle))
    glyph = Text(x="x", y="y", text="text", angle="angle",
        text_align="center", text_baseline="middle", text_font_size="11px")
    plot.add_glyph(text_source, glyph)


def to_base64(png):
    return "data:image/png;base64," + base64.b64encode(png).decode("utf-8")

urls = [ to_base64(icons.get(browser, b"")) for browser in browsers ]
x, y = polar_to_cartesian(1.7, start_angles, end_angles)

icons_source = ColumnDataSource(dict(urls=urls, x=x, y=y))
glyph = ImageURL(url="urls", x="x", y="y", anchor="center")
plot.add_glyph(icons_source, glyph)

text = [ "%.02f%%" % value for value in selected.Share ]
x, y = polar_to_cartesian(0.7, start_angles, end_angles)

text_source = ColumnDataSource(dict(text=text, x=x, y=y))
glyph = Text(x="x", y="y", text="text", text_align="center", text_baseline="middle")
plot.add_glyph(text_source, glyph)

doc = Document()
doc.add_root(plot)
doc.validate()

filename = "donut.html"
with open(filename, "w") as f:
    f.write(file_html(doc, INLINE, "Donut Chart"))
print("Wrote %s" % filename)
view(filename)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import subjects
from neuropy.utils import signal_process
sess = subjects.sd.ratSday3[0]

theta = sess.eegfile.get_signal(0)
filtered_theta = 